# DC Salvage Forecasting – Model Comparison
This notebook compares multiple forecasting models – ARIMA, SARIMAX, Prophet, and a placeholder for LSTM – to predict salvage volumes at a distribution center.
Each model is explained with **why it's used**, **when to use it**, and **evaluation results**.

In [12]:
pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
  Using cached pandas-1.5.3-cp39-cp39-macosx_11_0_arm64.whl (11.0 MB)
  Using cached numpy-1.24.3-cp39-cp39-macosx_11_0_arm64.whl (13.9 MB)
  Using cached matplotlib-3.7.1-cp39-cp39-macosx_11_0_arm64.whl (7.3 MB)
  Using cached seaborn-0.12.2-py3-none-any.whl (293 kB)
  Using cached scikit_learn-1.2.2-cp39-cp39-macosx_12_0_arm64.whl (8.5 MB)
  Using cached statsmodels-0.13.5-cp39-cp39-macosx_11_0_arm64.whl (9.2 MB)
     |████████████████████████████████| 8.2 MB 1.1 MB/s eta 0:00:01     |█▏                              | 307 kB 479 kB/s eta 0:00:17     |██▋                             | 675 kB 479 kB/s eta 0:00:16     |███████▎                        | 1.9 MB 479 kB/s eta 0:00:14
     |████████████████████████████████| 249 kB 13.5 MB/s eta 0:00:01
     |████████████████████████████████| 2.8 MB 497 kB/s eta 0:00:01    |▊                               | 61 kB 300 kB/s eta 0:00:09
     |██████████████████████████

In [16]:
pip uninstall prophet


Found existing installation: prophet 1.1.7
Uninstalling prophet-1.1.7:
  Would remove:
    /Users/Z00DW53/Library/Python/3.9/lib/python/site-packages/prophet-1.1.7.dist-info/*
    /Users/Z00DW53/Library/Python/3.9/lib/python/site-packages/prophet/*
Proceed (Y/n)? ^C
Note: you may need to restart the kernel to use updated packages.


In [18]:
pip install pystan==2.19.1.1
# pip install prophet==1.1.2


SyntaxError: invalid syntax (3497812171.py, line 1)

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from prophet import Prophet
from sklearn.metrics import mean_absolute_percentage_error

# Load data
df = pd.read_csv('salvage_data.csv', parse_dates=['week'])
df = df[(df['DC'] == 'DC123') & (df['Disposition'] == 'Salvage')].copy()
df.set_index('week', inplace=True)
df = df.asfreq('W')

df['Cube'].interpolate(inplace=True)

ModuleNotFoundError: No module named 'numpy.strings'

## ARIMA
**Why ARIMA?** Best for stationary time series with trend.

**When to use?** Use ARIMA when your data has no seasonality and is sufficiently long (>50 observations).

In [ ]:
train = df[:-12]['Cube']
test = df[-12:]['Cube']

arima_model = ARIMA(train, order=(1, 1, 1)).fit()
arima_forecast = arima_model.forecast(steps=12)
arima_mape = mean_absolute_percentage_error(test, arima_forecast)
print(f"ARIMA MAPE: {arima_mape:.2%}")

## SARIMAX
**Why SARIMAX?** Extends ARIMA with support for **seasonality** and **exogenous variables**.

**When to use?** Use SARIMAX when you have **weekly/monthly seasonality** or external regressors.

In [ ]:
sarimax_model = SARIMAX(train, order=(1, 1, 1), seasonal_order=(1, 1, 1, 52)).fit(disp=False)
sarimax_forecast = sarimax_model.forecast(steps=12)
sarimax_mape = mean_absolute_percentage_error(test, sarimax_forecast)
print(f"SARIMAX MAPE: {sarimax_mape:.2%}")

## Prophet
**Why Prophet?** Good for data with **multiple seasonalities** and **holiday effects**. Easy to tune.

**When to use?** Use Prophet when your time series has **daily/weekly/yearly patterns**, or when you want fast prototyping.

In [ ]:
prophet_df = df.reset_index()[['week', 'Cube']].rename(columns={'week': 'ds', 'Cube': 'y'})
prophet_train = prophet_df[:-12]

m = Prophet(weekly_seasonality=True, yearly_seasonality=True)
m.fit(prophet_train)

future = m.make_future_dataframe(periods=12, freq='W')
forecast = m.predict(future)

prophet_forecast = forecast.iloc[-12:]['yhat'].values
prophet_actual = prophet_df[-12:]['y'].values
prophet_mape = mean_absolute_percentage_error(prophet_actual, prophet_forecast)
print(f"Prophet MAPE: {prophet_mape:.2%}")

## LSTM (Placeholder)
**Why LSTM?** Captures complex temporal dependencies and works well for **non-linear** and **long memory** sequences.

**When to use?** Use LSTM when traditional statistical models fail to capture patterns, especially for high-frequency or multivariate data.

⚠️ Implementation is compute-intensive and is skipped for this demo.

In [ ]:
print("\nModel Comparison Summary:")
print(f"ARIMA MAPE: {arima_mape:.2%}")
print(f"SARIMAX MAPE: {sarimax_mape:.2%}")
print(f"Prophet MAPE: {prophet_mape:.2%}")